# Funciones

In [5]:
def do_schoolbook(f,g):
  n = len(f)
  m = len(g)
  C = [anillo(0)] * (n+m-1)
  for i in range(n):
    for j in range(m):
        C[i+j] = C[i+j] + f[i]*g[j]
  return C

def do_karatsuba(f,g, K_threshold=8):
  n = len(f)
  n0 = n //2
  n1 = n - n0
  if n < K_threshold:
    return do_schoolbook(f,g)
  f0 = f[:n0] # tamaño n0
  f1 = f[n0:]  # tamaño n1
  g0 = g[:n0] # tamaño n0
  g1 = g[n0:]  # tamaño n1
  u = do_karatsuba(f0,g0, K_threshold) # tamaño 2*n0-1
  v = do_karatsuba(f1,g1, K_threshold) # tamaño 2*n1-1
  # Usamos f1, g1 como f0+f1,g0+g1 para evitar crear más listas
  # Fijaos que n1 >= n0
  for i in range(n0):
    f1[i] = f1[i]+f0[i]
    g1[i] = g1[i]+g0[i]
  w = do_karatsuba(f1,g1, K_threshold) # tamaño 2*n1-1
  # resto u y v a w, el problema es que los tamaños pueden ser distintos
  for i in range(2*n0-1):
    w[i] = w[i] - u[i] - v[i]
  for i in range(2*n0-1,2*n1-1):
    w[i] = w[i] - v[i]
  #C = [0] * (2*n-1)
  #for i in range(2*n0-1):
  #  C[i] = C[i] + u[i]
  #  C[n0+i] = C[n0+i] + w[i]
  #  C[2*n0+i] = C[2*n0+i] + v[i]
  #for i in range(2*n0-1,2*n1-1):
  #  C[n0+i] = C[n0+i] + w[i]
  #  C[2*n0+i] = C[2*n0+i] + v[i]
  #
  # u contiene los monomios de 0 a 2*n0-2
  # v contiene los monomios de 2*n0 a 2*n-1
  # solo nos falta el monomio 2*n0-1
  # podemos usarlo para evitar sumas en C
  C = u +[anillo(0)] +v
  for i in range(2*n1-1):
    C[n0+i] = C[n0+i] + w[i]
  return C

def do_karatsuba_different_size(left: list, right: list, K_threshold: int=8) -> list:
    """
    Multiplicación de dos polinomios de diferente grado, usando una
    estrategia de división del polinómio mayor en partes de tamaño
    del polinomio menor. Así, poder aplicar do_karatsuba a las partes.

    INPUT:

    - ``left``  -- representación de polinomio como lista
    - ``right`` -- representación de polinomio como lista
    - ``K_threshold`` -- Entero, se usa como criterio para usar la 
    multiplicación de la escuela si el el grado de alguno de los 
    polinómios es menor que él.

    TESTS:

    sage: do_karatsuba_different_size([anillo(1), anillo(2)], [anillo(3), anillo(4)])  # Grados iguales
    [3, 10, 8]

    sage: do_karatsuba_different_size([anillo(1), anillo(2), anillo(3)], [anillo(4), anillo(5)])  # n > m
    [4, 13, 22, 15]

    sage: do_karatsuba_different_size([anillo(3), anillo(4)], [anillo(1), anillo(2), anillo(3)])  # n < m
    [3, 10, 17, 12]

    sage: do_karatsuba_different_size([], [anillo(1), anillo(2), anillo(3)])  # Caso vacío
    []

    sage: do_karatsuba_different_size([anillo(1)], [anillo(1), anillo(2), anillo(3)])  # Caso n = 1
    [1, 2, 3]

    sage: do_karatsuba_different_size([anillo(1), anillo(2), anillo(3)], [anillo(1)])  # Caso m = 1
    [1, 2, 3]

    sage: do_karatsuba_different_size([anillo(1), anillo(2), anillo(3)], [anillo(4), anillo(5)], K_threshold=1)  # Caso K_threshold bajo
    [4, 13, 22, 15]

    """
    n: int= len(left); m: int= len(right)
    if n == 0 or m == 0:
        return []
    if n == 1:
        c = left[0]
        return [c*a for a in right]
    if m == 1:
        c = right[0]
        return [a*c for a in left] # beware of noncommutative rings
    
    if n <= K_threshold or m <= K_threshold or K_threshold==1 or K_threshold==2:
        return do_schoolbook(left, right)
    if n == m:
        return do_karatsuba(left, right, K_threshold)
    if n > m:
        # left is the bigger list
        # n is the bigger number
        q = n // m
        r = n % m
        output = do_karatsuba(left[:m], right, K_threshold)
        for i in range(1, q): #from 1 <= i < q:
            mi = m*i
            carry = do_karatsuba(left[mi:mi+m], right, K_threshold)
            for j in range(m-1):
                output[mi+j] = output[mi+j] + carry[j]
            output.extend(carry[m-1:])
        if r:
            mi = m*q
            carry = do_karatsuba_different_size(left[mi:], right, K_threshold)
            for j in range (m-1):
                output[mi+j] = output[mi+j] + carry[j]
            output.extend(carry[m-1:])
        return output
    else:
        # n < m, I need to repeat the code due to the case
        # of noncommutative rings.
        q = m // n
        r = m % n
        output = do_karatsuba(left, right[:n], K_threshold)
        for i in range(1,q): #from 1 <= i < q:
            mi = n*i
            carry = do_karatsuba(left, right[mi:mi+n], K_threshold)
            for j in range(n-1):
                output[mi+j] = output[mi+j] + carry[j]
            output.extend(carry[n-1:])
        if r:
            mi = n*q
            carry = do_karatsuba_different_size(left, right[mi:], K_threshold)
            for j in range(n-1):
                output[mi+j] = output[mi+j] + carry[j]
            output.extend(carry[n-1:])
        return output

def do_karatsuba_different_size_mult_8(left: list, right: list, K_threshold: int=8) -> list:
    """
        Multiplicación de dos polinomios de diferente grado, usando una
        estrategia de división del polinómio mayor en partes de tamaño
        del polinomio menor. Así, poder aplicar do_karatsuba a las partes.

        INPUT:

        - ``left``  -- representación de polinomio como lista
        - ``right`` -- representación de polinomio como lista
        - ``K_threshold`` -- Entero, se usa como criterio para usar la 
        multiplicación de la escuela si el el grado de alguno de los 
        polinómios es menor que él.

        TESTS:

        sage: do_karatsuba_different_size([anillo(1), anillo(2)], [anillo(3), anillo(4)])  # Grados iguales
        [3, 10, 8]

        sage: do_karatsuba_different_size([anillo(1), anillo(2), anillo(3)], [anillo(4), anillo(5)])  # n > m
        [4, 13, 22, 15]

        sage: do_karatsuba_different_size([anillo(3), anillo(4)], [anillo(1), anillo(2), anillo(3)])  # n < m
        [3, 10, 17, 12]

        sage: do_karatsuba_different_size([], [anillo(1), anillo(2), anillo(3)])  # Caso vacío
        []

        sage: do_karatsuba_different_size([anillo(1)], [anillo(1), anillo(2), anillo(3)])  # Caso n = 1
        [1, 2, 3]

        sage: do_karatsuba_different_size([anillo(1), anillo(2), anillo(3)], [anillo(1)])  # Caso m = 1
        [1, 2, 3]

        sage: do_karatsuba_different_size([anillo(1), anillo(2), anillo(3)], [anillo(4), anillo(5)], K_threshold=1)  # Caso K_threshold bajo
        [4, 13, 22, 15]

    """
    n: int= len(left); m: int= len(right)
    if n == 0 or m == 0:
        return []
    if n == 1:
        c = left[0]
        return [c*a for a in right]
    if m == 1:
        c = right[0]
        return [a*c for a in left] # beware of noncommutative rings
    
    if n <= K_threshold or m <= K_threshold or K_threshold==1 or K_threshold==2:
        return do_schoolbook(left, right)
    if n == m:
        return do_karatsuba(left, right, K_threshold)

    # probablemnte sea mas eficiente ir sumando unos hasta qeu sea multiplo de 8
    if n % 8 !=0:
        n_prima = ((n+7)//8) * 8
        left = left + [anillo(0)]*(n_prima-n)
        n=n_prima
    if n > m:
        # left is the bigger list
        # n is the bigger number
        q = n // m
        r = n % m
        output = do_karatsuba(left[:m], right, K_threshold)
        for i in range(1, q): #from 1 <= i < q:
            mi = m*i
            carry = do_karatsuba(left[mi:mi+m], right, K_threshold)
            for j in range(m-1):
                output[mi+j] = output[mi+j] + carry[j]
            output.extend(carry[m-1:])
        if r:
            mi = m*q
            carry = do_karatsuba_different_size(left[mi:], right, K_threshold)
            for j in range (m-1):
                output[mi+j] = output[mi+j] + carry[j]
            output.extend(carry[m-1:])
        return output
    else:
        # n < m, I need to repeat the code due to the case
        # of noncommutative rings.
        q = m // n
        r = m % n
        output = do_karatsuba(left, right[:n], K_threshold)
        for i in range(1,q): #from 1 <= i < q:
            mi = n*i
            carry = do_karatsuba(left, right[mi:mi+n], K_threshold)
            for j in range(n-1):
                output[mi+j] = output[mi+j] + carry[j]
            output.extend(carry[n-1:])
        if r:
            mi = n*q
            carry = do_karatsuba_different_size(left, right[mi:], K_threshold)
            for j in range(n-1):
                output[mi+j] = output[mi+j] + carry[j]
            output.extend(carry[n-1:])
        return output

NUM_SUMA = 0
NUM_PRODUCTO = 0

class anillo:
    def __init__(self, valor):
        self.valor = valor

    def __repr__(self):
        return repr(self.valor)

    def __add__(self, otro):
        global NUM_SUMA 
        NUM_SUMA = NUM_SUMA + 1
        return anillo(self.valor + otro.valor)

    def __sub__(self,otro):
        global NUM_SUMA 
        NUM_SUMA = NUM_SUMA + 1
        return anillo(self.valor - otro.valor)

    def __mul__(self, otro):
        global NUM_PRODUCTO
        NUM_PRODUCTO = NUM_PRODUCTO + 1
        return anillo(self.valor * otro.valor)


# Ejercicio 1

Compare los métodos `do_karatsuba` y `do_schoolbook` vistos en clase para polinomios del mismo tamaño `n`.

- Construya sendas listas `L_K`, `L_S` tal que cada entrada sea un triplete `(i, n_p, n_o)` donde `i` sea el tamaño de los polinomios, `n_p` el número de productos que realiza cada algoritmo y `n_o` el número de operaciones totales (productos + sumas).

- Realice una representación gráfica del número de operaciones totales dependiendo del tamaño.

- Compruebe experimentalmente que `do_schoolbook` realiza $O(i^2)$ operaciones y `do_karatsuba` realiza $O(i^{\log_2(3)})$ operaciones.

- Experimente con el punto de corte (threshold) y comente sobre las gráficas resultantes para distintos puntos de corte.


## Generación de listas

In [5]:
def generador_listas(tamaño_max=100, card_cuerpo_finito=2, verbose=False):
    F=GF(card_cuerpo_finito)
    R=F['x']
    L_K=[]
    L_S=[]
    global NUM_SUMA, NUM_PRODUCTO    
    for i in range(tamaño_max+1):
        f = [anillo(R(1)) for t in range(i)]
        
        NUM_SUMA = 0
        NUM_PRODUCTO = 0
        k_result=do_karatsuba(f, f, K_threshold=8)
        L_K.append((i, NUM_PRODUCTO, NUM_SUMA+NUM_PRODUCTO))

        NUM_SUMA = 0
        NUM_PRODUCTO = 0
        s_result=do_schoolbook(f, f)
        L_S.append((i, NUM_PRODUCTO, NUM_SUMA+NUM_PRODUCTO))
        
        
        if verbose and i%50==0:
            print(i)        
    
    return L_K, L_S

In [ ]:
l_k, l_s = generador_listas(tamaño_max=1024*2, 
                            card_cuerpo_finito=2,
                            verbose=True)
print(l_k)
print(l_s)

In [ ]:
import pickle


def generador_listas2(funcion, tamaño_max=100, card_cuerpo_finito=2, K_threshold=8, export_pkl=False, verbose=False):
    F=GF(card_cuerpo_finito)
    R=F['x']
    L=[]
    g = [anillo(R(1))]*1024
    global NUM_SUMA, NUM_PRODUCTO 
    for i in range(tamaño_max+1):
        f = [anillo(R(1)) for t in range(i)]
        if funcion == 'do_karatsuba_different_size':
            NUM_SUMA = 0
            NUM_PRODUCTO = 0
            result=do_karatsuba_different_size(f, g, K_threshold=K_threshold)
            L.append((i, NUM_PRODUCTO, NUM_SUMA+NUM_PRODUCTO))
        elif funcion == 'do_schoolbook':
            NUM_SUMA = 0
            NUM_PRODUCTO = 0
            result=do_schoolbook(f, g)
            L.append((i, NUM_PRODUCTO, NUM_SUMA+NUM_PRODUCTO))
        else:
            raise ValueError('El valor de "funcion" debe ser "do_karatsuba_different_size" o "do_schoolbook"')
            break
        if verbose and i%50==0:
            print(i) 
    
    if export_pkl:
        if funcion == 'do_karatsuba_different_size':
            file_name = f'{funcion}_K({K_threshold})_{tamaño_max}.pkl'
        else:
            file_name = f'{funcion}_{tamaño_max}.pkl'
        if verbose:
            print(f'Exportando "{file_name}"...')
        with open(file_name, 'wb') as file:
            pickle.dump({'L': L}, file)
    return L

In [ ]:
l_kdiff= generador_listas2(funcion= 'do_karatsuba_different_size',
                            tamaño_max=2048, 
                            card_cuerpo_finito=2,
                            K_threshold=8,
                            export_pkl=True,
                            verbose=True)

print(l_kdiff)

l_sdiff= generador_listas2(funcion= 'do_schoolbook',
                            tamaño_max=2048, 
                            card_cuerpo_finito=2,
                            K_threshold=8,
                            export_pkl=True,
                            verbose=True)
print(l_sdiff)

### Guardado en archivos

In [7]:
import pickle
tam=len(l_k)-1
file_name = 'lk_ls_'+str(tam)+'T8.pkl'
with open(file_name, 'wb') as file:
    pickle.dump({'l_k': l_k, 'l_s': l_s}, file)


In [ ]:
open_file=file_name
with open(open_file, 'rb') as file:
    data = pickle.load(file)
    l_kr = data['l_k']
    l_sr = data['l_s']

print(l_kr)
print(l_sr)

## Representación Gráfica

In [ ]:
# Usando sage
line([(foo[0], foo[2]) for foo in l_s], color='blue', legend_label='Schoolbook') + line([(foo[0], foo[2]) for foo in l_k], color='red', legend_label='Karatsuba')

## Comprobación experimental Ordenes

In [ ]:
import matplotlib.pyplot as plt

def O_schoolbook_ops(n):
    return n**2  # O(n^2)

def O_karatsuba_ops(n):
    return n**(log(3)/log(2))  # O(n^log2(3))

sizes = range(len(l_k))
O_n2 = [2.1*O_schoolbook_ops(n) for n in sizes]
O_nlog23 = [12*O_karatsuba_ops(n) for n in sizes]

plt.figure(figsize=(10, 6))
plt.plot(sizes, O_n2, label="O(n^2)", linewidth=2)
plt.plot(sizes, [n[2] for n in l_s], label="Schoolbook", linewidth=2)
plt.plot(sizes, O_nlog23, label="O(n^log2(3))", linewidth=2)
plt.plot(sizes, [n[2] for n in l_k], label="Karatsuba", linewidth=2)
plt.xlabel("Tamaño del polinomio (n)")
plt.ylabel("Número de operaciones")
plt.title("Comparación de complejidad: Schoolbook vs Karatsuba")
plt.legend()
plt.grid()
plt.show()


## Experimentación Threshold

## Ejercicio 2

- Realice las mismas tareas que en el ejercicio anterior comparando `do_schoolbook` y la propuesta vista en clase de `do_karatsuba_different_size` para un polinomio $f$ de tamaño 1024 y un polinomio $g$ de tamaño variable $i$. Para esto considere un punto de corte de 8.

- Calcule para qué tamaños $i$ de $g$ nos encontramos con máximos locales que verifiquen:

$$
\text{NUM\_TOTAL}(1024, i) = \max\{\text{NUM\_TOTAL}(1024, k) : 1 \leq k \leq i\}
$$

$$
\exists j > i, \text{NUM\_TOTAL}(1024, j) < \text{NUM\_TOTAL}(1024, i)
$$

- Trate de modificar el código de `do_karatsuba_different_size` para que el tamaño de $g$ sea siempre un múltiplo de 8 añadiendo, si es preciso, ceros a la lista que representa $g$. ¿Cómo queda la gráfica con esta modificación? ¿Se ha mejorado? ¿A qué cree que se debe?


## Replicación ejercicio 1

In [2]:
def generador_listas(tamaño_max=100, card_cuerpo_finito=2, K_threshold=8, verbose=False):
    F=GF(card_cuerpo_finito)
    R=F['x']
    L_K=[]
    L_S=[]
    g = [anillo(R(1))]*1024
    global NUM_SUMA, NUM_PRODUCTO    
    for i in range(tamaño_max+1):
        f = [anillo(R(1)) for t in range(i)]
        NUM_SUMA = 0
        NUM_PRODUCTO = 0
        k_result=do_karatsuba_different_size(f, g, K_threshold=K_threshold)
        L_K.append((i, NUM_PRODUCTO, NUM_SUMA+NUM_PRODUCTO))

        NUM_SUMA = 0
        NUM_PRODUCTO = 0
        s_result=do_schoolbook(f, g)
        L_S.append((i, NUM_PRODUCTO, NUM_SUMA+NUM_PRODUCTO))
        
        
        if verbose and i%50==0:
            print(i)        
    
    return L_K, L_S



In [ ]:
l_k, l_s = generador_listas(tamaño_max=2048, 
                            card_cuerpo_finito=2,
                            K_threshold=8,
                            verbose=True)

print(l_k)
print(l_s)

## Calcular tamaños de i que cumplan la condición

## Modificación do_karatsuba_different_size tamaño g multiplo de 8

In [71]:
def multiplo_mayor(g):
    return ((g+7)// 8) * 8


In [ ]:
multiplo_mayor(32)

# EXEC

In [9]:
def do_karatsuba_different_size_mult_8(left: list, right: list, K_threshold: int=8) -> list:
    """
        Multiplicación de dos polinomios de diferente grado, usando una
        estrategia de división del polinómio mayor en partes de tamaño
        del polinomio menor. Así, poder aplicar do_karatsuba a las partes.

        INPUT:

        - ``left``  -- representación de polinomio como lista
        - ``right`` -- representación de polinomio como lista
        - ``K_threshold`` -- Entero, se usa como criterio para usar la 
        multiplicación de la escuela si el el grado de alguno de los 
        polinómios es menor que él.

        TESTS:

        sage: do_karatsuba_different_size([anillo(1), anillo(2)], [anillo(3), anillo(4)])  # Grados iguales
        [3, 10, 8]

        sage: do_karatsuba_different_size([anillo(1), anillo(2), anillo(3)], [anillo(4), anillo(5)])  # n > m
        [4, 13, 22, 15]

        sage: do_karatsuba_different_size([anillo(3), anillo(4)], [anillo(1), anillo(2), anillo(3)])  # n < m
        [3, 10, 17, 12]

        sage: do_karatsuba_different_size([], [anillo(1), anillo(2), anillo(3)])  # Caso vacío
        []

        sage: do_karatsuba_different_size([anillo(1)], [anillo(1), anillo(2), anillo(3)])  # Caso n = 1
        [1, 2, 3]

        sage: do_karatsuba_different_size([anillo(1), anillo(2), anillo(3)], [anillo(1)])  # Caso m = 1
        [1, 2, 3]

        sage: do_karatsuba_different_size([anillo(1), anillo(2), anillo(3)], [anillo(4), anillo(5)], K_threshold=1)  # Caso K_threshold bajo
        [4, 13, 22, 15]

    """
    n: int= len(left); m: int= len(right)
    if n == 0 or m == 0:
        return []
    if n == 1:
        c = left[0]
        return [c*a for a in right]
    if m == 1:
        c = right[0]
        return [a*c for a in left] # beware of noncommutative rings
    
    if n <= K_threshold or m <= K_threshold or K_threshold==1 or K_threshold==2:
        return do_schoolbook(left, right)
    if n == m:
        return do_karatsuba(left, right, K_threshold)

    # Probablemnte sea mas eficiente ir sumando unos hasta qeu sea multiplo de 8
    if n % 8 !=0:
        n_prima = ((n+7)//8) * 8
        left = left + [anillo(0)]*(n_prima-n)
        n=n_prima
    if n > m:
        # left is the bigger list
        # n is the bigger number
        q = n // m
        r = n % m
        output = do_karatsuba(left[:m], right, K_threshold)
        for i in range(1, q): #from 1 <= i < q:
            mi = m*i
            carry = do_karatsuba(left[mi:mi+m], right, K_threshold)
            for j in range(m-1):
                output[mi+j] = output[mi+j] + carry[j]
            output.extend(carry[m-1:])
        if r:
            mi = m*q
            carry = do_karatsuba_different_size(left[mi:], right, K_threshold)
            for j in range (m-1):
                output[mi+j] = output[mi+j] + carry[j]
            output.extend(carry[m-1:])
        return output
    else:
        # n < m, I need to repeat the code due to the case
        # of noncommutative rings.
        q = m // n
        r = m % n
        output = do_karatsuba(left, right[:n], K_threshold)
        for i in range(1,q): #from 1 <= i < q:
            mi = n*i
            carry = do_karatsuba(left, right[mi:mi+n], K_threshold)
            for j in range(n-1):
                output[mi+j] = output[mi+j] + carry[j]
            output.extend(carry[n-1:])
        if r:
            mi = n*q
            carry = do_karatsuba_different_size(left, right[mi:], K_threshold)
            for j in range(n-1):
                output[mi+j] = output[mi+j] + carry[j]
            output.extend(carry[n-1:])
        return output

In [10]:
import pickle


def generador_listas(funcion, diff_size, tamaño_max=100, card_cuerpo_finito=2, K_threshold=8, export_pkl=False, verbose=False):
    F=GF(card_cuerpo_finito)
    R=F['x']
    L=[]
    if diff_size:
        g = [anillo(R(1))]*1024

    global NUM_SUMA, NUM_PRODUCTO 
    for i in range(tamaño_max+1):
        f = [anillo(R(1)) for t in range(i)]
        if not diff_size:
            g=f

        if funcion == 'do_karatsuba' and diff_size:
            NUM_SUMA = 0
            NUM_PRODUCTO = 0
            result=do_karatsuba_different_size(f, g, K_threshold=K_threshold)
            L.append((i, NUM_PRODUCTO, NUM_SUMA+NUM_PRODUCTO))
        
        elif funcion == 'do_karatsuba_mult8' and diff_size:
            NUM_SUMA = 0
            NUM_PRODUCTO = 0
            result=do_karatsuba_different_size_mult_8(f, g, K_threshold=K_threshold)
            L.append((i, NUM_PRODUCTO, NUM_SUMA+NUM_PRODUCTO))
        elif funcion == 'do_karatsuba':
            NUM_SUMA = 0
            NUM_PRODUCTO = 0
            result=do_karatsuba(f, g, K_threshold=K_threshold)
            L.append((i, NUM_PRODUCTO, NUM_SUMA+NUM_PRODUCTO))

        elif funcion == 'do_schoolbook':
            NUM_SUMA = 0
            NUM_PRODUCTO = 0
            result=do_schoolbook(f, g)
            L.append((i, NUM_PRODUCTO, NUM_SUMA+NUM_PRODUCTO))
        else:
            raise ValueError('El valor de "funcion" debe ser "do_karatsuba", "do_schoolbook", "do_karatsuba_mult8" y ojo con el "diff_size"')
            break
        if verbose and i%50==0:
            print(i) 
    
    if export_pkl:
        if funcion == 'do_karatsuba_different_size':
            file_name = f'{funcion}_K({K_threshold})_{tamaño_max}.pkl'
        else:
            file_name = f'{funcion}_different_size_{tamaño_max}.pkl'
        if verbose:
            print(f'Exportando "{file_name}"...')
        with open(file_name, 'wb') as file:
            pickle.dump({'L': L}, file)
    return L

In [12]:
L_K_MULT= generador_listas(funcion= 'do_karatsuba_mult8',
                            diff_size=True,
                            tamaño_max=2048, 
                            card_cuerpo_finito=2,
                            K_threshold=8,
                            export_pkl=True,
                            verbose=True)

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100


KeyboardInterrupt: 